In [1]:
# Agrege acá el código para importar las librerias
from shutil import copyfile

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [2]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...

# Se copian los archivos para renombrarlos sin borrar los originales
filename = 'Evaluación del Pitch (2021.04.13 ISoftware).csv'.encode('utf-8')
copyfile(filename, 'evaluacion.csv')
copyfile('Asistencia del Pitch (2021.04.13 ISoftware).csv', 'asistencia.csv')

#Se cargan los datos a HDFS, carpeta tmp
!hdfs dfs -copyFromLocal asistencia.csv /tmp/
!hdfs dfs -copyFromLocal evaluacion.csv /tmp/

# Se borran los nuevos archivos
!rm 'evaluacion.csv'
!rm 'asistencia.csv'

In [3]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...
asistencia = SesionSpark.read.load('/tmp/asistencia.csv',
                                    format = 'csv',
                                    inferSchema = True,
                                    header = True )

evaluacion = SesionSpark.read.load('/tmp/evaluacion.csv',
                                    format = 'csv',
                                    inferSchema = True,
                                    header = True )

asistencia.createOrReplaceTempView('vista_Asistencia')
evaluacion.createOrReplaceTempView('vista_Evaluacion')

In [4]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...
ContextoSql.sql("""SELECT DISTINCT campo2 AS email 
                     FROM vista_Asistencia
                     WHERE campo2 IS NOT NULL
                     ORDER BY email""").show()

+--------------------+
|               email|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
|anoriega@unal.edu.co|
|auarbelaeza@unal....|
| bocampo@unal.edu.co|
|cgiraldo@unal.edu.co|
|cjfunezg@unal.edu.co|
|cquinchiar@unal.e...|
| dadazam@unal.edu.co|
|daestradam@unal.e...|
|davgarciava@unal....|
|dballesteroso@una...|
|  dbrito@unal.edu.co|
|dcadavid@unal.edu.co|
|dcardonaal@unal.e...|
|dchavarriar@unal....|
|dgarciabl@unal.ed...|
+--------------------+
only showing top 20 rows



In [63]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...
#Cantidad de asistencias por estudiante
ContextoSql.sql("""SELECT campo2 AS email,
                          COUNT(campo2) AS cantidad_veces,
                          MIN(campo1) AS primera_asistencia,
                          MAX(campo1) AS ultima_asistencia
                    FROM vista_Asistencia
                    WHERE campo2 IS NOT NULL
                    GROUP BY email""").show()

+--------------------+--------------+--------------------+--------------------+
|               email|cantidad_veces|  primera_asistencia|   ultima_asistencia|
+--------------------+--------------+--------------------+--------------------+
|samolinap@unal.ed...|            10|2021/04/08 8:11:4...|2021/04/13 9:37:2...|
| jruedat@unal.edu.co|             8|2021/04/08 8:12:5...|2021/04/13 9:41:2...|
|nvalenciat@unal.e...|             8|2021/04/08 8:10:1...|2021/04/13 9:39:3...|
|serendona@unal.ed...|             8|2021/04/08 8:10:4...|2021/04/13 9:40:2...|
|jgutierrezlo@unal...|             8|2021/04/08 8:12:0...|2021/04/13 9:39:0...|
|dgerenal@unal.edu.co|             8|2021/04/08 8:10:0...|2021/04/13 9:41:2...|
|cjfunezg@unal.edu.co|             8|2021/04/08 8:12:1...|2021/04/13 9:41:4...|
|emflorezb@unal.ed...|             8|2021/04/08 8:14:0...|2021/04/13 9:45:4...|
|dchavarriar@unal....|             8|2021/04/08 8:13:0...|2021/04/13 9:43:5...|
|fguerrerot@unal.e...|             8|202

In [45]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...
# No se tiene en cuenta la asistencia a la presentación de su mismo grupo
ContextoSql.sql("""SELECT v1.campo2 AS email,
                          COUNT(v1.campo2) AS cantidad_veces
                    FROM vista_Asistencia AS v1 INNER JOIN vista_Asistencia AS v2
                        ON v1.campo1 = v2.campo1 AND v1.campo2 = v2.campo2
                        AND SUBSTRING(v1.campo3,15,100) <> SUBSTRING(v2.campo4,9,100)
                    GROUP BY email
                    HAVING cantidad_veces <= 2""").show()

+--------------------+--------------+
|               email|cantidad_veces|
+--------------------+--------------+
|jdroldano@unal.ed...|             1|
+--------------------+--------------+



In [51]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...
# No se tiene en cuenta la asistencia a la presentación de su mismo grupo
ContextoSql.sql("""SELECT v1.campo2 AS email,
                          COUNT(v1.campo2) AS cantidad_veces
                    FROM vista_Asistencia AS v1 INNER JOIN vista_Asistencia AS v2
                        ON v1.campo1 = v2.campo1 AND v1.campo2 = v2.campo2
                        AND SUBSTRING(v1.campo3,15,100) <> SUBSTRING(v2.campo4,9,100)
                    GROUP BY email
                    HAVING cantidad_veces = 0""").show()

+-----+--------------+
|email|cantidad_veces|
+-----+--------------+
+-----+--------------+



In [62]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...
ContextoSql.sql("""SELECT DISTINCT SUBSTRING(campo3,15,100) AS equipo,
                          campo2 AS email                   
                    FROM vista_Asistencia
                    WHERE campo3 IS NOT NULL
                    ORDER BY equipo, email""").show()

+--------------------+--------------------+
|              equipo|               email|
+--------------------+--------------------+
|Administración de...|angutierrezb@unal...|
|Administración de...| eapenad@unal.edu.co|
|Administración de...|emflorezb@unal.ed...|
|Administración de...|framirezf@unal.ed...|
|Administración de...|jhcordoba@unal.ed...|
|Administración de...| jortize@unal.edu.co|
|Administración de...| kmolano@unal.edu.co|
|Administración de...|lumarinb@unal.edu.co|
|Administración de...|nsalinas@unal.edu.co|
|Administración de...|rmbuilesm@unal.ed...|
|Administración de...|saespinosab@unal....|
|Administración de...|samolinap@unal.ed...|
|Administración de...|sednarvaezna@unal...|
|Gestión de Menús ...| abanolc@unal.edu.co|
|Gestión de Menús ...|anapariciom@unal....|
|Gestión de Menús ...|davgarciava@unal....|
|Gestión de Menús ...|emlopezr@unal.edu.co|
|Gestión de Menús ...|favasquezj@unal.e...|
|Gestión de Menús ...|hriveraa@unal.edu.co|
|Gestión de Menús ...|jcardonaso

In [69]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...
ContextoSql.sql("""SELECT SUBSTRING(v1.campo4,9,100) AS exposicion,
                          COUNT(v1.campo2) AS asistentes
                    FROM vista_Asistencia AS v1 INNER JOIN vista_Asistencia AS v2
                        ON v1.campo1 = v2.campo1 AND v1.campo2 = v2.campo2
                        AND SUBSTRING(v1.campo3,15,100) <> SUBSTRING(v2.campo4,9,100)
                    GROUP BY exposicion""").show()

+--------------------+----------+
|          exposicion|asistentes|
+--------------------+----------+
|Servicios a domic...|        76|
|Lo tengo y te lo ...|        75|
|Gestión de Reside...|        72|
|       Te lo cambio.|        75|
|Administración de...|        70|
|Deserción de estu...|         3|
|Gestión de Menús ...|        72|
|Gestión de progra...|        74|
|     Mi Sanandresito|        72|
+--------------------+----------+



In [73]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...
ContextoSql.sql("""SELECT v1.campo2 AS email,
                          SUBSTRING(v1.campo3,15,100),
                          v2.campo3
                    FROM vista_Asistencia AS v1 INNER JOIN vista_Evaluacion AS v2
                        ON v1.campo2 = v2.campo2 AND SUBSTRING(v1.campo3,15,100) = v2.campo3""").show()

+--------------------+--------------------------+--------------------+
|               email|substring(campo3, 15, 100)|              campo3|
+--------------------+--------------------------+--------------------+
|daestradam@unal.e...|      Lo tengo y te lo ...|Lo tengo y te lo ...|
|anapariciom@unal....|      Servicios a domic...|Servicios a domic...|
|hriveraa@unal.edu.co|      Gestión de Menús ...|Gestión de Menús ...|
| jruedat@unal.edu.co|      Servicios a domic...|Servicios a domic...|
|  lmayar@unal.edu.co|      Gestión de progra...|Gestión de progra...|
|daestradam@unal.e...|      Lo tengo y te lo ...|Lo tengo y te lo ...|
| jruedat@unal.edu.co|      Servicios a domic...|Servicios a domic...|
|hriveraa@unal.edu.co|      Gestión de Menús ...|Gestión de Menús ...|
| kmolano@unal.edu.co|      Administración de...|Administración de...|
|jhcordoba@unal.ed...|      Administración de...|Administración de...|
|  lmayar@unal.edu.co|      Gestión de progra...|Gestión de progra...|
|daest

In [61]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...
ContextoSql.sql("""SELECT campo3 AS equipo,
                          ROUND(AVG(campo4),2) AS Introduccion,
                          ROUND(AVG(campo5),2) AS Equipo,
                          ROUND(AVG(campo6),2) AS Problema,
                          ROUND(AVG(campo7),2) AS Ventajas,
                          ROUND(AVG(campo8),2) AS Solucion,
                          ROUND(AVG(campo9),2) AS Producto,
                          ROUND(AVG(campo10),2) AS Traccion,
                          ROUND(AVG(campo11),2) AS Mercado,
                          ROUND(AVG(campo12),2) AS Competencia,
                          ROUND(AVG(campo13),2) AS Modelo_Negocio,
                          ROUND(AVG(campo14),2) AS Inversion,
                          ROUND(AVG(campo15),2) AS Contacto,
                          ROUND(AVG(campo16),2) AS Coordinacion,
                          ROUND(AVG(campo17),2) AS Claridad,
                          ROUND(AVG(campo18),2) AS Diapositivas,
                          ROUND((ROUND(AVG(campo4),2)+
                                 ROUND(AVG(campo5),2)+
                                 ROUND(AVG(campo6),2)+
                                 ROUND(AVG(campo7),2)+
                                 ROUND(AVG(campo8),2)+
                                 ROUND(AVG(campo9),2)+
                                 ROUND(AVG(campo10),2)+
                                 ROUND(AVG(campo11),2)+
                                 ROUND(AVG(campo12),2)+
                                 ROUND(AVG(campo13),2)+
                                 ROUND(AVG(campo14),2)+
                                 ROUND(AVG(campo15),2)+
                                 ROUND(AVG(campo16),2)+
                                 ROUND(AVG(campo17),2)+
                                 ROUND(AVG(campo18),2))/15,2) AS Promedio_Total
                     FROM vista_Evaluacion
                     WHERE campo3 IS NOT NULL
                     GROUP BY equipo""").show()

+--------------------+------------+------+--------+--------+--------+--------+--------+-------+-----------+--------------+---------+--------+------------+--------+------------+--------------+
|              equipo|Introduccion|Equipo|Problema|Ventajas|Solucion|Producto|Traccion|Mercado|Competencia|Modelo_Negocio|Inversion|Contacto|Coordinacion|Claridad|Diapositivas|Promedio_Total|
+--------------------+------------+------+--------+--------+--------+--------+--------+-------+-----------+--------------+---------+--------+------------+--------+------------+--------------+
| permitiendo hace...|        null|  null|    null|    null|    null|    null|    null|   null|       null|          null|     null|    null|        null|    null|        null|          null|
|Servicios a domic...|         4.5|  4.64|    4.53|    4.28|    4.51|    4.61|    4.13|   4.14|       4.33|          4.28|     4.17|    4.65|        4.49|    4.56|        4.56|          4.43|
| podría ser perde...|        null|  nul

In [60]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...

#Introduccion (Utilizando ORDER BY DESC y LIMIT)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo4),2) AS Introduccion
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Introduccion = ( SELECT ROUND(AVG(campo4),2) AS Introduccion
                                                 FROM vista_Evaluacion
                                                 WHERE campo3 IS NOT NULL
                                                 GROUP BY campo3
                                                 ORDER BY Introduccion DESC 
                                                 LIMIT 1 ) """).show()

#Equipo (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo5),2) AS Equipo_Prom
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Equipo_Prom = ( SELECT MAX(sub.Equipo_Prom)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo5),2) AS Equipo_Prom
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Problema (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo6),2) AS Problema
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Problema = ( SELECT MAX(sub.Problema)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo6),2) AS Problema
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Ventajas (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo7),2) AS Ventajas
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Ventajas = ( SELECT MAX(sub.Ventajas)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo7),2) AS Ventajas
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Solucion (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo8),2) AS Solucion
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Solucion = ( SELECT MAX(sub.Solucion)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo8),2) AS Solucion
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Producto (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo9),2) AS Producto
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Producto = ( SELECT MAX(sub.Producto)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo9),2) AS Producto
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Traccion (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo10),2) AS Traccion
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Traccion = ( SELECT MAX(sub.Traccion)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo10),2) AS Traccion
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Mercado (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo11),2) AS Mercado
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Mercado = ( SELECT MAX(sub.Mercado)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo11),2) AS Mercado
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Competencia (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo12),2) AS Competencia
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Competencia = ( SELECT MAX(sub.Competencia)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo12),2) AS Competencia
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Modelo_Negocio (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo13),2) AS Modelo_Negocio
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Modelo_Negocio = ( SELECT MAX(sub.Modelo_Negocio)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo13),2) AS Modelo_Negocio
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Inversion (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo14),2) AS Inversion
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Inversion = ( SELECT MAX(sub.Inversion)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo14),2) AS Inversion
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Contacto (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo15),2) AS Contacto
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Contacto = ( SELECT MAX(sub.Contacto)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo15),2) AS Contacto
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Coordinacion (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo16),2) AS Coordinacion
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Coordinacion = ( SELECT MAX(sub.Coordinacion)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo16),2) AS Coordinacion
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Claridad (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo17),2) AS Claridad
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Claridad = ( SELECT MAX(sub.Claridad)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo17),2) AS Claridad
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Diapositivas (Utilizando MAX)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo18),2) AS Diapositivas
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Diapositivas = ( SELECT MAX(sub.Diapositivas)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo18),2) AS Diapositivas
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Mayor promedio total  (Utilizando ORDER BY DESC y LIMIT)
ContextoSql.sql("""SELECT campo3 AS equipo,
                          ROUND((ROUND(AVG(campo4),2)+
                                 ROUND(AVG(campo5),2)+
                                 ROUND(AVG(campo6),2)+
                                 ROUND(AVG(campo7),2)+
                                 ROUND(AVG(campo8),2)+
                                 ROUND(AVG(campo9),2)+
                                 ROUND(AVG(campo10),2)+
                                 ROUND(AVG(campo11),2)+
                                 ROUND(AVG(campo12),2)+
                                 ROUND(AVG(campo13),2)+
                                 ROUND(AVG(campo14),2)+
                                 ROUND(AVG(campo15),2)+
                                 ROUND(AVG(campo16),2)+
                                 ROUND(AVG(campo17),2)+
                                 ROUND(AVG(campo18),2))/15,2) AS Mejor_Promedio_Total
                         FROM vista_Evaluacion
                         WHERE campo3 IS NOT NULL
                         GROUP BY equipo
                         ORDER BY Mejor_Promedio_Total DESC
                         LIMIT 1""").show()

+--------------------+------------+
|              equipo|Introduccion|
+--------------------+------------+
|Gestión de Reside...|        4.55|
+--------------------+------------+

+--------------------+-----------+
|              equipo|Equipo_Prom|
+--------------------+-----------+
|Deserción de estu...|        5.0|
+--------------------+-----------+

+--------------------+--------+
|              equipo|Problema|
+--------------------+--------+
|Deserción de estu...|     5.0|
+--------------------+--------+

+--------------------+--------+
|              equipo|Ventajas|
+--------------------+--------+
|Deserción de estu...|     5.0|
+--------------------+--------+

+--------------------+--------+
|              equipo|Solucion|
+--------------------+--------+
|Servicios a domic...|    4.51|
+--------------------+--------+

+--------------------+--------+
|              equipo|Producto|
+--------------------+--------+
|Deserción de estu...|     5.0|
+--------------------+--------+


In [59]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...

#Introduccion (Utilizando ORDER BY ASC, HAVING y LIMIT)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo4),2) AS Introduccion
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Introduccion = ( SELECT ROUND(AVG(campo4),2) AS Introduccion
                                                 FROM vista_Evaluacion
                                                 WHERE campo3 IS NOT NULL
                                                 GROUP BY campo3
                                                 HAVING Introduccion IS NOT NULL
                                                 ORDER BY Introduccion ASC
                                                 LIMIT 1 ) """).show()

#Equipo (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo5),2) AS Equipo_Prom
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Equipo_Prom = ( SELECT MIN(sub.Equipo_Prom)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo5),2) AS Equipo_Prom
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Problema (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo6),2) AS Problema
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Problema = ( SELECT MIN(sub.Problema)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo6),2) AS Problema
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Ventajas (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo7),2) AS Ventajas
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Ventajas = ( SELECT MIN(sub.Ventajas)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo7),2) AS Ventajas
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Solucion (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo8),2) AS Solucion
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Solucion = ( SELECT MIN(sub.Solucion)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo8),2) AS Solucion
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Producto (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo9),2) AS Producto
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Producto = ( SELECT MIN(sub.Producto)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo9),2) AS Producto
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Traccion (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo10),2) AS Traccion
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Traccion = ( SELECT MIN(sub.Traccion)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo10),2) AS Traccion
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Mercado (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo11),2) AS Mercado
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Mercado = ( SELECT MIN(sub.Mercado)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo11),2) AS Mercado
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Competencia (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo12),2) AS Competencia
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Competencia = ( SELECT MIN(sub.Competencia)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo12),2) AS Competencia
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Modelo_Negocio (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo13),2) AS Modelo_Negocio
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Modelo_Negocio = ( SELECT MIN(sub.Modelo_Negocio)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo13),2) AS Modelo_Negocio
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Inversion (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo14),2) AS Inversion
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Inversion = ( SELECT MIN(sub.Inversion)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo14),2) AS Inversion
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Contacto (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo15),2) AS Contacto
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Contacto = ( SELECT MIN(sub.Contacto)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo15),2) AS Contacto
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Coordinacion (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo16),2) AS Coordinacion
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Coordinacion = ( SELECT MIN(sub.Coordinacion)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo16),2) AS Coordinacion
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Claridad (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo17),2) AS Claridad
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Claridad = ( SELECT MIN(sub.Claridad)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo17),2) AS Claridad
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Diapositivas (Utilizando MIN)
ContextoSql.sql(""" SELECT campo3 AS equipo, ROUND(AVG(campo18),2) AS Diapositivas
                      FROM vista_Evaluacion
                      GROUP BY equipo
                      HAVING Diapositivas = ( SELECT MIN(sub.Diapositivas)
                                               FROM ( SELECT campo3 AS equipo,
                                                             ROUND(AVG(campo18),2) AS Diapositivas
                                                      FROM vista_Evaluacion
                                                      WHERE campo3 IS NOT NULL
                                                      GROUP BY equipo ) AS sub ) """).show()

#Mayor promedio total  (Utilizando ORDER BY ASC, HAVING y LIMIT)
ContextoSql.sql("""SELECT campo3 AS equipo,
                          ROUND((ROUND(AVG(campo4),2)+
                                 ROUND(AVG(campo5),2)+
                                 ROUND(AVG(campo6),2)+
                                 ROUND(AVG(campo7),2)+
                                 ROUND(AVG(campo8),2)+
                                 ROUND(AVG(campo9),2)+
                                 ROUND(AVG(campo10),2)+
                                 ROUND(AVG(campo11),2)+
                                 ROUND(AVG(campo12),2)+
                                 ROUND(AVG(campo13),2)+
                                 ROUND(AVG(campo14),2)+
                                 ROUND(AVG(campo15),2)+
                                 ROUND(AVG(campo16),2)+
                                 ROUND(AVG(campo17),2)+
                                 ROUND(AVG(campo18),2))/15,2) AS Peor_Promedio_Total
                         FROM vista_Evaluacion
                         WHERE campo3 IS NOT NULL
                         GROUP BY equipo
                         HAVING Peor_Promedio_Total IS NOT NULL
                         ORDER BY Peor_Promedio_Total ASC
                         LIMIT 1""").show()

+--------------------+------------+
|              equipo|Introduccion|
+--------------------+------------+
|Administración de...|        4.13|
+--------------------+------------+

+---------------+-----------+
|         equipo|Equipo_Prom|
+---------------+-----------+
|Mi Sanandresito|        3.9|
+---------------+-----------+

+--------------------+--------+
|              equipo|Problema|
+--------------------+--------+
|Administración de...|    3.97|
+--------------------+--------+

+--------------------+--------+
|              equipo|Ventajas|
+--------------------+--------+
|Gestión de progra...|    3.92|
+--------------------+--------+

+--------------------+--------+
|              equipo|Solucion|
+--------------------+--------+
|Administración de...|    4.19|
+--------------------+--------+

+--------------------+--------+
|              equipo|Producto|
+--------------------+--------+
|Gestión de Menús ...|    4.03|
+--------------------+--------+

+--------------------+--

In [54]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...
ContextoSql.sql("""SELECT campo3 AS equipo,
                          ROUND((ROUND(AVG(campo4),2)+
                                 ROUND(AVG(campo5),2)+
                                 ROUND(AVG(campo6),2)+
                                 ROUND(AVG(campo7),2)+
                                 ROUND(AVG(campo8),2)+
                                 ROUND(AVG(campo9),2)+
                                 ROUND(AVG(campo10),2)+
                                 ROUND(AVG(campo11),2)+
                                 ROUND(AVG(campo12),2)+
                                 ROUND(AVG(campo13),2)+
                                 ROUND(AVG(campo14),2)+
                                 ROUND(AVG(campo15),2)+
                                 ROUND(AVG(campo16),2)+
                                 ROUND(AVG(campo17),2)+
                                 ROUND(AVG(campo18),2))/15,2) AS Promedio_Total
                         FROM vista_Evaluacion
                         WHERE campo3 IS NOT NULL 
                         GROUP BY equipo
                         ORDER BY Promedio_Total DESC""").show()

+--------------------+--------------+
|              equipo|Promedio_Total|
+--------------------+--------------+
|Deserción de estu...|          4.77|
|Lo tengo y te lo ...|          4.48|
|Gestión de Reside...|          4.47|
|Servicios a domic...|          4.43|
|Gestión de progra...|           4.4|
|Gestión de Menús ...|          4.36|
|       Te lo cambio.|          4.28|
|     Mi Sanandresito|           4.2|
|Administración de...|          4.19|
| permitiendo hace...|          null|
| tracción es qué ...|          null|
| podría ser perde...|          null|
| no se requiere y...|          null|
| pero es por falt...|          null|
| su propuesta de ...|          null|
| y/o en el caso d...|          null|
+--------------------+--------------+

